<a href="https://colab.research.google.com/github/N-Linh/ML/blob/main/softmax_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [60]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import datasets
iris_data = datasets.load_iris()

In [61]:
print(iris_data.DESCR)

.. _iris_dataset:

Iris plants dataset
--------------------

**Data Set Characteristics:**

    :Number of Instances: 150 (50 in each of three classes)
    :Number of Attributes: 4 numeric, predictive attributes and the class
    :Attribute Information:
        - sepal length in cm
        - sepal width in cm
        - petal length in cm
        - petal width in cm
        - class:
                - Iris-Setosa
                - Iris-Versicolour
                - Iris-Virginica
                
    :Summary Statistics:

    ============== ==== ==== ======= ===== ====================
                    Min  Max   Mean    SD   Class Correlation
    ============== ==== ==== ======= ===== ====================
    sepal length:   4.3  7.9   5.84   0.83    0.7826
    sepal width:    2.0  4.4   3.05   0.43   -0.4194
    petal length:   1.0  6.9   3.76   1.76    0.9490  (high!)
    petal width:    0.1  2.5   1.20   0.76    0.9565  (high!)
    ============== ==== ==== ======= ===== ===========

In [62]:
X = iris_data["data"][:, (2, 3)]  # petal length, petal width
y = iris_data["target"]
X.shape, y.shape

((150, 2), (150,))

using batch gradient descent

In [63]:
X_ones = np.ones([len(X), 1])
X_com = np.c_[X_ones, X]
X_com.shape

(150, 3)

In [65]:
np.random.seed(2022)

train test split function implementation

In [66]:
test_ratio = 0.2
val_ratio = 0.2
size = len(X)
test_size = int(test_ratio*size)
val_size = int(size*val_ratio)
train_size = size - test_size - val_size

random_index = np.random.permutation(len(X))

X_train = X_com[random_index[:train_size]]
y_train = y[random_index[:train_size]]
X_val = X_com[random_index[train_size: -test_size]]
y_val = y[random_index[train_size:-test_size]]
X_test = X_com[random_index[-test_size:]]
y_test = y[random_index[-test_size:]]
X_train.shape, X_val.shape, X_test.shape

((90, 3), (30, 3), (30, 3))

assign label by one-hot

In [67]:
def one_hot(y):
  num_classes = max(y) + 1
  num_size = len(y)
  y_one_hot = np.zeros((num_size, num_classes))
  y_one_hot[np.arange(num_size), y] = 1
  return y_one_hot

In [68]:
y_train[:5]

array([2, 2, 0, 2, 0])

In [69]:
one_hot(y_train[:5])

array([[0., 0., 1.],
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.]])

In [70]:
Y_train_one_hot = one_hot(y_train)
Y_val_one_hot = one_hot(y_val)
Y_test_one_hot = one_hot(y_test)

soft max function

In [71]:
def softmax(logits):#logits = [p1, p2, p3, ..., pc]
  exps_logits = np.exp(logits)#[exp(p1), exp(p2), exp(p3), ..., exp(pc)]
  sum_exps = np.sum(exps_logits, axis=1, keepdims = True)
  return exps_logits/sum_exps

In [72]:
n_inputs = X_train.shape[1]#contain bias
n_outputs = len(np.unique(y))
n_inputs, n_outputs

(3, 3)

In [73]:
eta = 0.01
n_iterations = 5001
m = len(X_train)
epsilon = 1e-7

Theta = np.random.randn(n_inputs, n_outputs)

for iteration in range(n_iterations):
    logits = X_train.dot(Theta)
    Y_proba = softmax(logits)
    if iteration % 500 == 0:
        loss = -np.mean(np.sum(Y_train_one_hot * np.log(Y_proba + epsilon), axis=1))#+epsilon, log(p_k) not exist if p_k = 0, avoid nan
        print(iteration, loss)
    error = Y_proba - Y_train_one_hot
    gradients = 1/m * X_train.T.dot(error)
    Theta = Theta - eta * gradients

0 0.8157074684889276
500 0.6390622800526229
1000 0.5577217477574312
1500 0.5037087062042496
2000 0.46482058346387745
2500 0.43508203507968246
3000 0.41130585346587
3500 0.39165349533899424
4000 0.3749918485025087
4500 0.36058323560677585
5000 0.34792514583900364


In [74]:
Theta

array([[ 4.11662423,  0.26609363, -1.8291646 ],
       [-1.2245585 , -0.1394312 , -0.71801808],
       [-2.43336691, -1.05020608,  2.05271346]])

predict val data

In [75]:
logits = X_val.dot(Theta)
Y_proba = softmax(logits)
y_predict = np.argmax(Y_proba, axis=1)

accuracy_score = np.mean(y_predict == y_val)
accuracy_score

0.8666666666666667

using regulariztion and early stopping, compute loss on val data and stop when loss increases

In [76]:
eta = 0.1 
n_iterations = 5001
m = len(X_train)
epsilon = 1e-7
alpha = 0.1  # regularization hyperparameter
best_loss = np.infty

Theta = np.random.randn(n_inputs, n_outputs)

for iteration in range(n_iterations):
    logits = X_train.dot(Theta)
    Y_proba = softmax(logits)
    error = Y_proba - Y_train_one_hot
    gradients = 1/m * X_train.T.dot(error) + np.r_[np.zeros([1, n_outputs]), alpha * Theta[1:]]
    Theta = Theta - eta * gradients

    logits = X_val.dot(Theta)
    Y_proba = softmax(logits)
    xentropy_loss = -np.mean(np.sum(Y_val_one_hot * np.log(Y_proba + epsilon), axis=1))
    l2_loss = 1/2 * np.sum(np.square(Theta[1:]))
    loss = xentropy_loss + alpha * l2_loss
    if iteration % 500 == 0:
        print(iteration, loss)
    if loss < best_loss:
        best_loss = loss
    else:
        print(iteration - 1, best_loss)
        print(iteration, loss, "early stopping!")
        break

0 1.0244271935246383
500 0.5909130067516503
1000 0.5726383938499379
1500 0.5698865645807576
1593 0.5698481390629014
1594 0.5698481459628099 early stopping!


In [77]:
logits = X_val.dot(Theta)
Y_proba = softmax(logits)
y_predict = np.argmax(Y_proba, axis=1)

accuracy_score = np.mean(y_predict == y_val )
accuracy_score

0.9

In [78]:
logits = X_test.dot(Theta)
Y_proba = softmax(logits)
y_predict = np.argmax(Y_proba, axis=1)

accuracy_score = np.mean(y_predict == y_test)
accuracy_score

0.9333333333333333